# 1. Data input

In [5]:
import csv

questionaire_input = []

# read statistical.csv
with open('statistical.csv',encoding = 'utf-8',newline = '') as csvfile:
     reader = csv.reader(csvfile,delimiter = ',')
     for row in reader:
        questionaire_input.append(row)

# 2. Data preprocessing

In [6]:
# use one-hot encoding to transform the string to number in each feature

# make dictionary for encoding(not include tolerance label)
feature_dict = {}
for i in range(0,14):
    feature_dict[questionaire_input[0][i]] = {}

for i in range(1,len(questionaire_input)):
    for j in range(0,14):
        feature_now = questionaire_input[0][j] # the feature of current element
        if not isinstance(questionaire_input[i][j],int): # just used for test
            if questionaire_input[i][j] not in feature_dict[feature_now]:
        
                # use difference number to identify each value
                feature_dict[feature_now][questionaire_input[i][j]] = len(feature_dict[feature_now]) 
                questionaire_input[i][j] = feature_dict[feature_now][questionaire_input[i][j]] # change the value type of feature from str to int
            else:
                questionaire_input[i][j] = feature_dict[feature_now][questionaire_input[i][j]]

# list the dictionary of one-hot encoding below
print("one hot encoding dictionary\n")
for key in feature_dict:
    print(key," ",feature_dict[key])


one hot encoding dictionary

gender   {'女': 0, '男': 1}
age   {'16～18歲': 0, '22～28歲': 1, '19～22歲': 2, '29歲以上': 3, '15歲以下': 4}
department   {'ee': 0, 'ed': 1, 'lb': 2, 'bi': 3, 'oe': 4, 'sc': 5, 'cs': 6, 'mg': 7, 'ot': 8}
total_usetime   {'三至四小時': 0, '五小時以上': 1, '二至三小時': 2, '一小時以下': 3, '一至二小時': 4, '一至兩小時': 5}
social_usetime   {'半至一小時': 0, '半小時以下': 1, '二至三小時': 2, '一至二小時': 3, '三小時以上': 4, '': 5}
communication_usetime   {'半至一小時': 0, '三小時以上': 1, '一至二小時': 2, '': 3, '二至三小時': 4, '半小時以下': 5}
entertainment_usetime   {'半至一小時': 0, '一至二小時': 1, '半小時以下': 2, '二至三小時': 3, '三小時以上': 4, '': 5}
news_usetime   {'半小時以下': 0, '半至一小時': 1, '': 2, '一至二小時': 3, '三小時以上': 4}
system_usetime   {'半小時以下': 0, '': 1, '半至一小時': 2, '一至二小時': 3}
notification_usetime   {'半小時以下': 0, '半至一小時': 1, '': 2, '一至二小時': 3, '二至三小時': 4}
view_order_top   {'最先看': 0, '最後看': 1, '中': 2}
view_order_middle   {'中': 0, '最後看': 1, '最先看': 2}
view_order_bottom   {'最後看': 0, '中': 1, '最先看': 2}
scenario   {'working': 0, 'resting': 1, 'commuting': 2, 'wakingup':

# 3. Model construction

參數:
宣告SVM model時
svm_model_name = svm.SVC(C = float(number),kernel = 'kernel_name', gamma = float(number))
p.s : SVM會使用不同的kernel function, 我用一個dictionary存放4個不同kernel function的SVM，名稱是svm_dict

validation參數
1. holdout validation : test ratio = 0.125

2. k-fold validation : k = 2

SVM模型參數
1. C = 100 (僅開放rbf和poly，用在linear和sigmoid會跑很久)

2. gamma = 0.1(僅開放rbf和poly(大概要跑6小時)，gamma為初始值時跑比較快(30分鐘)，用在linear和sigmoid會跑很久)
  其他參數則為初始值

觀察:

1. 無論使用的validation，rbf為kernel function時通常會有最高的accuracy(有時polynomial會佔據最高accuracy)

2. 當SVM中的參數 C = 100.0 時有最好的整體accuracy(約能提升1%到3%)，整體accuracy在C變大和變小時遞減

3. 當SVM中的參數 gamma = 0.1 時有最好的整體accuracy

4. SVM的參數gamma對整體的accuracy的提升影響微弱，甚至會降低一些accuracy

p.s : C和gamma的參數調控只限於rbf和polynomial，linear和sigmoid不適用(你可以試試看，但我試了之後跑了一個晚上加一個上午還沒跑完，然後gamma=0.1時用於rbf和poly時用了大概6小時跑完)


In [7]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import math
import time

# try 4 different SVM model to test the difference
svm_dict = {'linear' : svm.SVC(kernel='linear'),
            'rbf' : svm.SVC(C=100,kernel='rbf',gamma=0.1),
            'poly' : svm.SVC(C=100,kernel='poly',gamma=0.1),
            'sigmoid': svm.SVC(kernel='sigmoid')}

# change the data to numpy array
questionaire_input = np.array(questionaire_input[1:])
np.random.shuffle(questionaire_input)

# choose the test ratio and k with highest accuracy with given test times

test_time = 10
test_ratio = 0.125
k = 2


# accuracy for each tolerance 
accuracy_111111 = {'linear':0,
                   'poly':0,
                   'rbf':0,
                   'sigmoid':0}

accuracy_1122 = {'linear':0,
                 'poly':0,
                 'rbf':0,
                 'sigmoid':0}

accuracy_123 = {'linear':0,
                'poly':0,
                'rbf':0,
                'sigmoid':0}

accuracy_222 = {'linear':0,
                'poly':0,
                'rbf':0,
                'sigmoid':0}

accuracy_24 = {'linear':0,
               'poly':0,
               'rbf':0,
               'sigmoid':0}

accuracy_15 = {'linear':0,
               'poly':0,
               'rbf':0,
               'sigmoid':0}

# confusion matrix
'''
confusion_hold_111111 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_hold_1122 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_hold_123 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_hold_222 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_hold_24 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_hold_15 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}

confusion_kfold_111111 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_kfold_1122 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_kfold_123 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_kfold_222 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_kfold_24 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
confusion_kfold_15 = {'linear':[],'poly':[],'rbf':[],'sigmoid':[]}
'''


start_time = time.time()


# X and y for each model
X_111111 = questionaire_input[1:,0:14]
y_111111 = questionaire_input[1:,14]

X_1122 = questionaire_input[1:,0:14]
y_1122 = questionaire_input[1:,15]

X_123 = questionaire_input[1:,0:14]
y_123 = questionaire_input[1:,16]

X_222 = questionaire_input[1:,0:14]
y_222 = questionaire_input[1:,17]

X_24 = questionaire_input[1:,0:14]
y_24 = questionaire_input[1:,18]

X_15 = questionaire_input[1:,0:14]
y_15 = questionaire_input[1:,19]

print("holdout validation with test ratio = 0.125\n")
for i in range(test_time):
    # holdout data
    train_x_111111, test_x_111111, train_y_1111111, test_y_111111 = train_test_split(X_111111,y_111111,test_size = 0.125)
    train_x_1122, test_x_1122, train_y_1122, test_y_1122 = train_test_split(X_1122,y_1122,test_size = 0.125)
    train_x_123, test_x_123, train_y_123, test_y_123 = train_test_split(X_123,y_123,test_size = 0.125)
    train_x_222, test_x_222, train_y_222, test_y_222 = train_test_split(X_222,y_222,test_size = 0.125)
    train_x_24, test_x_24, train_y_24, test_y_24 = train_test_split(X_24,y_24,test_size = 0.125)
    train_x_15, test_x_15, train_y_15, test_y_15 = train_test_split(X_15,y_15,test_size = 0.125)

    
    for key in svm_dict:
        # 1 1 1 1 1 1
        svm_dict[key].fit(train_x_111111,train_y_1111111)
        pred_y_111111 = svm_dict[key].predict(test_x_111111)
        accuracy_111111[key] += accuracy_score(test_y_111111, pred_y_111111)
        #confusion_hold_111111[key] += confusion_matrix(test_y_111111, pred_y_111111)


        # 1 1 2 2
        svm_dict[key].fit(train_x_1122,train_y_1122)
        pred_y_1122 = svm_dict[key].predict(test_x_1122)
        accuracy_1122[key] += accuracy_score(test_y_1122, pred_y_1122)
        #confusion_hold_1122[key] += confusion_matrix(test_y_1122, pred_y_1122)

        # 1 2 3
        svm_dict[key].fit(train_x_123,train_y_123)
        pred_y_123 = svm_dict[key].predict(test_x_123)
        accuracy_123[key] += accuracy_score(test_y_123, pred_y_123)
        #confusion_hold_123[key] += confusion_matrix(test_y_123, pred_y_123)

        # 2 2 2
        svm_dict[key].fit(train_x_222,train_y_222)
        pred_y_222 = svm_dict[key].predict(test_x_222)
        accuracy_222[key] += accuracy_score(test_y_222, pred_y_222)
        #confusion_hold_123[key] += confusion_matrix(test_y_222, pred_y_222)

        # 2 4
        svm_dict[key].fit(train_x_24,train_y_24)
        pred_y_24 = svm_dict[key].predict(test_x_24)
        accuracy_24[key] += accuracy_score(test_y_24, pred_y_24)
        #confusion_hold_24[key] += confusion_matrix(test_y_24, pred_y_24)

        # 1 5
        svm_dict[key].fit(train_x_15,train_y_15)
        pred_y_15 = svm_dict[key].predict(test_x_15)
        accuracy_15[key] += accuracy_score(test_y_15, pred_y_15)
        #confusion_hold_15[key] += confusion_matrix(test_y_15, pred_y_15)





        '''
        if key == 'rbf':
            for i in range(len(svm_rbf_test)):
                for j in range(6):
                    if j == 0:
                        svm_rbf_test[i].fit(train_x_111111,train_y_1111111)
                        accuracy_rbf_test[i][j] += accuracy_score(test_y_111111,svm_rbf_test[i].predict(test_x_111111)) 
                    if j == 1:
                        svm_rbf_test[i].fit(train_x_1122,train_y_1122)
                        accuracy_rbf_test[i][j] += accuracy_score(test_y_1122,svm_rbf_test[i].predict(test_x_1122)) 
                    if j == 2:
                        svm_rbf_test[i].fit(train_x_123,train_y_123)
                        accuracy_rbf_test[i][j] += accuracy_score(test_y_123,svm_rbf_test[i].predict(test_x_123)) 
                    if j == 3:
                        svm_rbf_test[i].fit(train_x_222,train_y_222)
                        accuracy_rbf_test[i][j] += accuracy_score(test_y_222,svm_rbf_test[i].predict(test_x_222))
                    if j == 4:
                        svm_rbf_test[i].fit(train_x_24,train_y_24)
                        accuracy_rbf_test[i][j] += accuracy_score(test_y_24,svm_rbf_test[i].predict(test_x_24))
                    if j == 5:
                        svm_rbf_test[i].fit(train_x_15,train_y_15)
                        accuracy_rbf_test[i][j] += accuracy_score(test_y_15,svm_rbf_test[i].predict(test_x_15))


        if key == 'poly':
            for i in range(len(svm_poly_test)):
                for j in range(6):
                    if j == 0:
                        svm_poly_test[i].fit(train_x_111111,train_y_1111111)
                        accuracy_poly_test[i][j] += accuracy_score(test_y_111111,svm_poly_test[i].predict(test_x_111111)) 
                    if j == 1:
                        svm_poly_test[i].fit(train_x_1122,train_y_1122)
                        accuracy_poly_test[i][j] += accuracy_score(test_y_1122,svm_poly_test[i].predict(test_x_1122)) 
                    if j == 2:
                        svm_poly_test[i].fit(train_x_123,train_y_123)
                        accuracy_poly_test[i][j] += accuracy_score(test_y_123,svm_poly_test[i].predict(test_x_123)) 
                    if j == 3:
                        svm_poly_test[i].fit(train_x_222,train_y_222)
                        accuracy_poly_test[i][j] += accuracy_score(test_y_222,svm_poly_test[i].predict(test_x_222))
                    if j == 4:
                        svm_poly_test[i].fit(train_x_24,train_y_24)
                        accuracy_poly_test[i][j] += accuracy_score(test_y_24,svm_poly_test[i].predict(test_x_24))
                    if j == 5:
                        svm_poly_test[i].fit(train_x_15,train_y_15)
                        accuracy_poly_test[i][j] += accuracy_score(test_y_15,svm_poly_test[i].predict(test_x_15))
        '''



'''
for key in svm_dict:
    confusion_hold_111111[key] /= test_time
    confusion_hold_1122[key] /= test_time
    confusion_hold_123[key] /= test_time
    confusion_hold_222[key] /= test_time
    confusion_hold_24[key] /= test_time
    confusion_hold_15[key] /= test_time
'''
# print the accuracy of holdout
print("========================")

print("  accuracy of 1 1 1 1 1 1:")
for key in accuracy_111111:
    print("    ",key," : ",accuracy_111111[key]/test_time,"\n")

print("========================")


print("  accuracy of 1 1 2 2:")
for key in accuracy_1122:
    print("    ",key," : ",accuracy_1122[key]/test_time,"\n")

print("========================")


print("  accuracy of 1 2 3:")
for key in accuracy_123:
    print("    ",key," : ",accuracy_123[key]/test_time,"\n")

print("========================")


print("  accuracy of 2 2 2:")
for key in accuracy_222:
    print("    ",key," : ",accuracy_222[key]/test_time,"\n")

print("========================")


print("  accuracy of 2 4:")
for key in accuracy_24:
    print("    ",key," : ",accuracy_24[key]/test_time,"\n")

print("========================")


print("  accuracy of 1 5:")
for key in accuracy_15:
    print("    ",key," : ",accuracy_15[key]/test_time,"\n")

print("========================")


# print the kernel with highest probability
max_proba_kernel_111111 = "linear"
for key in accuracy_111111:
    if accuracy_111111[max_proba_kernel_111111] < accuracy_111111[key]:
        max_proba_kernel_111111 = key

print("the highest accuracy of 111111 : ",accuracy_111111[max_proba_kernel_111111]/test_time,"with kernel : ",max_proba_kernel_111111)

max_proba_kernel_1122 = "linear"
for key in accuracy_1122:
    if accuracy_1122[max_proba_kernel_1122] < accuracy_1122[key]:
        max_proba_kernel_1122 = key

print("the highest accuracy of 1122 : ",accuracy_1122[max_proba_kernel_1122]/test_time,"with kernel : ",max_proba_kernel_1122)


max_proba_kernel_123 = "linear"
for key in accuracy_123:
    if accuracy_123[max_proba_kernel_123] < accuracy_123[key]:
        max_proba_kernel_123 = key

print("the highest accuracy of 123 : ",accuracy_123[max_proba_kernel_123]/test_time,"with kernel : ",max_proba_kernel_123)


max_proba_kernel_222 = "linear"
for key in accuracy_222:
    if accuracy_222[max_proba_kernel_222] < accuracy_222[key]:
        max_proba_kernel_222 = key

print("the highest accuracy of 222 : ",accuracy_222[max_proba_kernel_222]/test_time,"with kernel : ",max_proba_kernel_222)


max_proba_kernel_24 = "linear"
for key in accuracy_24:
    if accuracy_24[max_proba_kernel_24] < accuracy_24[key]:
        max_proba_kernel_24 = key

print("the highest accuracy of 24 : ",accuracy_24[max_proba_kernel_24]/test_time,"with kernel : ",max_proba_kernel_24)


max_proba_kernel_15 = "linear"
for key in accuracy_15:
    if accuracy_15[max_proba_kernel_15] < accuracy_15[key]:
        max_proba_kernel_15 = key

print("the highest accuracy of 15 : ",accuracy_15[max_proba_kernel_15]/test_time,"with kernel : ",max_proba_kernel_15)


'''
# print confusion matrix
plt.figure(figsize = (10,10))
ax_hold_111111 = sn.heatmap(confusion_hold_111111[max_proba_kernel_111111])
ax_hold_111111.set_title("1 1 1 1 1 1 with test ratio = 0.125 & kernel = ",max_proba_kernel_111111)
plt.show()

plt.figure(figsize = (10,10))
ax_hold_1122 = sn.heatmap(confusion_hold_1122[max_proba_kernel_1122])
ax_hold_1122.set_title("1 1 2 2 with test ratio = 0.125 & kernel = ",max_proba_kernel_1122)
plt.show()

plt.figure(figsize = (10,10))
ax_hold_123 = sn.heatmap(confusion_hold_123[max_proba_kernel_123])
ax_hold_123.set_title("1 2 3 with test ratio = 0.125 & kernel = ",max_proba_kernel_123)
plt.show()

plt.figure(figsize = (10,10))
ax_hold_222 = sn.heatmap(confusion_hold_222[max_proba_kernel_222])
ax_hold_222.set_title("2 2 2 with test ratio = 0.125 & kernel = ",max_proba_kernel_222)
plt.show()

plt.figure(figsize = (10,10))
ax_hold_24 = sn.heatmap(confusion_hold_24[max_proba_kernel_24])
ax_hold_24.set_title("2 4 with test ratio = 0.125 & kernel = ",max_proba_kernel_24)
plt.show()

plt.figure(figsize = (10,10))
ax_hold_15 = sn.heatmap(confusion_hold_15[max_proba_kernel_15])
ax_hold_15.set_title("1 5 with test ratio = 0.125 & kernel = ",max_proba_kernel_15)
plt.show()
'''



print("========================\n")

print("k-fold validation with k = 2\n")

# k-fold
for i in range(1,len(questionaire_input),k):

    # k-fold data for 1 1 1 1 1 1
    train_x_now_111111 = np.concatenate((questionaire_input[:i,0:14],questionaire_input[i+k:,0:14]))
    train_y_now_111111 = np.concatenate((questionaire_input[:i,14],questionaire_input[i+k:,14]))
    test_x_now_111111 = questionaire_input[i:i+k,0:14]
    test_y_now_111111 = questionaire_input[i:i+k,14]

    for key in svm_dict:
        svm_dict[key].fit(train_x_now_111111,train_y_now_111111)
        pred_y_111111 = svm_dict[key].predict(test_x_now_111111)
        accuracy_111111[key] += accuracy_score(test_y_now_111111,pred_y_111111)
        #confusion_kfold_111111[key] += confusion_matrix(test_y_now_111111,pred_y_111111)



    # k-fold data for 1 1 2 2 
    train_x_now_1122 = np.concatenate((questionaire_input[:i,0:14],questionaire_input[i+k:,0:14]))
    train_y_now_1122 = np.concatenate((questionaire_input[:i,15],questionaire_input[i+k:,15]))
    test_x_now_1122 = questionaire_input[i:i+k,0:14]
    test_y_now_1122 = questionaire_input[i:i+k,15]

    for key in svm_dict:
        svm_dict[key].fit(train_x_now_1122,train_y_now_1122)
        pred_y_1122 = svm_dict[key].predict(test_x_now_1122)
        accuracy_1122[key] += accuracy_score(test_y_now_1122,pred_y_1122)
        #confusion_kfold_1122[key] += confusion_matrix(test_y_now_1122,pred_y_1122)

    # k-fold data for 1 2 3
    train_x_now_123 = np.concatenate((questionaire_input[:i,0:14],questionaire_input[i+k:,0:14]))
    train_y_now_123 = np.concatenate((questionaire_input[:i,16],questionaire_input[i+k:,16]))
    test_x_now_123 = questionaire_input[i:i+k,0:14]
    test_y_now_123 = questionaire_input[i:i+k,16]

    for key in svm_dict:
        svm_dict[key].fit(train_x_now_123,train_y_now_123)
        pred_y_123 = svm_dict[key].predict(test_x_now_123)
        accuracy_123[key] += accuracy_score(test_y_now_123,pred_y_123)
        #confusion_kfold_123[key] += confusion_matrix(test_y_now_123,pred_y_123)


    # k-fold data for 2 2 2 
    train_x_now_222 = np.concatenate((questionaire_input[:i,0:14],questionaire_input[i+k:,0:14]))
    train_y_now_222 = np.concatenate((questionaire_input[:i,17],questionaire_input[i+k:,17]))
    test_x_now_222 = questionaire_input[i:i+k,0:14]
    test_y_now_222 = questionaire_input[i:i+k,17]

    for key in svm_dict:
        svm_dict[key].fit(train_x_now_222,train_y_now_222)
        pred_y_222 = svm_dict[key].predict(test_x_now_222)
        accuracy_222[key] += accuracy_score(test_y_now_222,pred_y_222)
        #confusion_kfold_222[key] += confusion_matrix(test_y_now_222,pred_y_222)

    # k-fold data for 2 4 
    train_x_now_24 = np.concatenate((questionaire_input[:i,0:14],questionaire_input[i+k:,0:14]))
    train_y_now_24 = np.concatenate((questionaire_input[:i,18],questionaire_input[i+k:,18]))
    test_x_now_24 = questionaire_input[i:i+k,0:14]
    test_y_now_24 = questionaire_input[i:i+k,18]

    for key in svm_dict:
        svm_dict[key].fit(train_x_now_24,train_y_now_24)
        pred_y_24 = svm_dict[key].predict(test_x_now_24)
        accuracy_24[key] += accuracy_score(test_y_now_24,pred_y_24)
        #confusion_kfold_24[key] += confusion_matrix(test_y_now_24,pred_y_24)

        

    # k-fold data for 1 5 
    train_x_now_15 = np.concatenate((questionaire_input[:i,0:14],questionaire_input[i+k:,0:14]))
    train_y_now_15 = np.concatenate((questionaire_input[:i,19],questionaire_input[i+k:,19]))
    test_x_now_15 = questionaire_input[i:i+k,0:14]
    test_y_now_15 = questionaire_input[i:i+k,19]

    for key in svm_dict:
        svm_dict[key].fit(train_x_now_15,train_y_now_15)
        pred_y_15 = svm_dict[key].predict(test_x_now_15)
        accuracy_15[key] += accuracy_score(test_y_now_15,pred_y_15)
        #confusion_kfold_15[key] += confusion_matrix(test_y_now_15,pred_y_15)
'''
for key in svm_dict:
    confusion_kfold_111111[key] /= k
    confusion_kfold_1122[key] /= k
    confusion_kfold_123[key] /= k
    confusion_kfold_222[key] /= k
    confusion_kfold_24[key] /= k
    confusion_kfold_15[key] /= k
'''
print("\n\n")


print("========================")

print("  accuracy of 1 1 1 1 1 1:")
for key in accuracy_111111:
    print("    ",key," : ",accuracy_111111[key]/int(len(questionaire_input)/k),"\n")

print("========================")


print("  accuracy of 1 1 2 2:")
for key in accuracy_1122:
    print("    ",key," : ",accuracy_1122[key]/int(len(questionaire_input)/k),"\n")

print("========================")


print("  accuracy of 1 2 3:")
for key in accuracy_123:
    print("    ",key," : ",accuracy_123[key]/int(len(questionaire_input)/k),"\n")

print("========================")


print("  accuracy of 2 2 2:")
for key in accuracy_222:
    print("    ",key," : ",accuracy_222[key]/int(len(questionaire_input)/k),"\n")

print("========================")


print("  accuracy of 2 4:")
for key in accuracy_24:
    print("    ",key," : ",accuracy_24[key]/int(len(questionaire_input)/k),"\n")

print("========================")


print("  accuracy of 1 5:")
for key in accuracy_15:
    print("    ",key," : ",accuracy_15[key]/int(len(questionaire_input)/k),"\n")

print("========================")

# print the kernel with highest probability
max_proba_kernel_111111 = "linear"
for key in accuracy_111111:
    if accuracy_111111[max_proba_kernel_111111] < accuracy_111111[key]:
        max_proba_kernel_111111 = key

print("the highest accuracy of 111111 : ",accuracy_111111[max_proba_kernel_111111]/int(len(questionaire_input)/k),"with kernel : ",max_proba_kernel_111111)

max_proba_kernel_1122 = "linear"
for key in accuracy_1122:
    if accuracy_1122[max_proba_kernel_1122] < accuracy_1122[key]:
        max_proba_kernel_1122 = key

print("the highest accuracy of 1122 : ",accuracy_1122[max_proba_kernel_1122]/int(len(questionaire_input)/k),"with kernel : ",max_proba_kernel_1122)


max_proba_kernel_123 = "linear"
for key in accuracy_123:
    if accuracy_123[max_proba_kernel_123] < accuracy_123[key]:
        max_proba_kernel_123 = key

print("the highest accuracy of 123 : ",accuracy_123[max_proba_kernel_123]/int(len(questionaire_input)/k),"with kernel : ",max_proba_kernel_123)


max_proba_kernel_222 = "linear"
for key in accuracy_222:
    if accuracy_222[max_proba_kernel_222] < accuracy_222[key]:
        max_proba_kernel_222 = key

print("the highest accuracy of 222 : ",accuracy_222[max_proba_kernel_222]/int(len(questionaire_input)/k),"with kernel : ",max_proba_kernel_222)


max_proba_kernel_24 = "linear"
for key in accuracy_24:
    if accuracy_24[max_proba_kernel_24] < accuracy_24[key]:
        max_proba_kernel_24 = key

print("the highest accuracy of 24 : ",accuracy_24[max_proba_kernel_24]/int(len(questionaire_input)/k),"with kernel : ",max_proba_kernel_24)


max_proba_kernel_15 = "linear"
for key in accuracy_15:
    if accuracy_15[max_proba_kernel_15] < accuracy_15[key]:
        max_proba_kernel_15 = key

print("the highest accuracy of 15 : ",accuracy_15[max_proba_kernel_15]/int(len(questionaire_input)/k),"with kernel : ",max_proba_kernel_15)

'''
print("========================")


for i in range(len(accuracy_rbf_test)):
    for j in range(6):
        print(accuracy_rbf_test[i][0]/test_time," ",accuracy_rbf_test[i][1]/test_time," ",accuracy_rbf_test[i][2]/test_time," ",accuracy_rbf_test[i][3]/test_time," ",accuracy_rbf_test[i][4]/test_time," ",accuracy_rbf_test[i][5]/test_time)


print("\n ======================== \n")
for i in range(len(accuracy_poly_test)):
    for j in range(6):
        print(accuracy_poly_test[i][0]/test_time," ",accuracy_poly_test[i][1]/test_time," ",accuracy_poly_test[i][2]/test_time," ",accuracy_poly_test[i][3]/test_time," ",accuracy_poly_test[i][4]/test_time," ",accuracy_poly_test[i][5]/test_time)
'''


finish_time = time.time() - start_time
print("program execution time : ",finish_time)

holdout validation with test ratio = 0.125

  accuracy of 1 1 1 1 1 1:
     linear  :  0.14528301886792452 

     poly  :  0.19905660377358492 

     rbf  :  0.23867924528301887 

     sigmoid  :  0.04433962264150943 

  accuracy of 1 1 2 2:
     linear  :  0.15849056603773584 

     poly  :  0.2490566037735849 

     rbf  :  0.2971698113207547 

     sigmoid  :  0.10471698113207546 

  accuracy of 1 2 3:
     linear  :  0.21037735849056607 

     poly  :  0.3169811320754717 

     rbf  :  0.3584905660377358 

     sigmoid  :  0.1330188679245283 

  accuracy of 2 2 2:
     linear  :  0.22830188679245284 

     poly  :  0.28773584905660377 

     rbf  :  0.31132075471698106 

     sigmoid  :  0.16226415094339625 

  accuracy of 2 4:
     linear  :  0.440566037735849 

     poly  :  0.4490566037735849 

     rbf  :  0.4518867924528303 

     sigmoid  :  0.35094339622641507 

  accuracy of 1 5:
     linear  :  0.5132075471698112 

     poly  :  0.5877358490566038 

     rbf  :  0.60849056